In [18]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
#read in data/maryam/ranking_results_borda.csv
df = pd.read_csv('../data/tables/analysis/ranking-results-kemney-young.csv')
df

,Sidewalk barriers,Walking cane,Walker,Mobility scooter,Manual wheelchair,Motorized wheelchair
0,Uneven sidewalk panels,2.13,2.32,2.81,6.57,1.99
1,Broken surface or cracks on sidewalk,3.11,1.82,3.93,7.10,6.13
2,Manholes on sidewalk,7.22,7.54,8.38,8.61,7.05
3,Brick or cobblestone surface,6.78,6.77,8.97,7.95,7.93
4,Grass surface,6.00,7.15,7.80,7.12,8.84
5,Sand or gravel surface,3.80,4.67,6.56,4.07,6.65
6,Narrow sidewalk (width less than 1.2m / 4 ft),4.30,2.99,5.73,5.56,4.65
7,Missing curb ramp at intersections,4.38,1.56,1.71,2.45,1.79
8,Steep slope (sidewalk incline greater than 5%),1.63,4.75,5.76,3.52,5.66


In [20]:
#create a dicttionary with barriers and index
barrier_index = dict(zip(df['Sidewalk barriers'], range(len(df['Sidewalk barriers']))))

In [21]:
#apply barrier index to the dataframe
df['barrier_index'] = df['Sidewalk barriers'].map(barrier_index)

In [22]:
# turn scores into ranks for each mobility aid user
for column in ['Walking cane', 'Walker', 'Mobility scooter', 'Manual wheelchair', 'Motorized wheelchair']:
    df[column + '_rank'] = df[column].rank(method='min') - 1

In [23]:
df

,Sidewalk barriers,Walking cane,Walker,Mobility scooter,Manual wheelchair,Motorized wheelchair,barrier_index,Walking cane_rank,Walker_rank,Mobility scooter_rank,Manual wheelchair_rank,Motorized wheelchair_rank
0,Uneven sidewalk panels,2.13,2.32,2.81,6.57,1.99,0,1.0,2.0,1.0,4.0,1.0
1,Broken surface or cracks on sidewalk,3.11,1.82,3.93,7.10,6.13,1,2.0,1.0,2.0,5.0,4.0
2,Manholes on sidewalk,7.22,7.54,8.38,8.61,7.05,2,8.0,8.0,7.0,8.0,6.0
3,Brick or cobblestone surface,6.78,6.77,8.97,7.95,7.93,3,7.0,6.0,8.0,7.0,7.0
4,Grass surface,6.00,7.15,7.80,7.12,8.84,4,6.0,7.0,6.0,6.0,8.0
5,Sand or gravel surface,3.80,4.67,6.56,4.07,6.65,5,3.0,4.0,5.0,2.0,5.0
6,Narrow sidewalk (width less than 1.2m / 4 ft),4.30,2.99,5.73,5.56,4.65,6,4.0,3.0,3.0,3.0,2.0
7,Missing curb ramp at intersections,4.38,1.56,1.71,2.45,1.79,7,5.0,0.0,0.0,0.0,0.0
8,Steep slope (sidewalk incline greater than 5%),1.63,4.75,5.76,3.52,5.66,8,0.0,5.0,4.0,1.0,3.0


In [24]:
from scipy.stats import mannwhitneyu

# Prepare the ranks for each group
groups = ['Walking cane_rank', 'Walker_rank', 'Mobility scooter_rank', 'Manual wheelchair_rank', 'Motorized wheelchair_rank']
group_data = {group: df.sort_values(by=group, ascending=False)['barrier_index'].tolist() for group in groups}

# Perform the Mann-Whitney U test between all pairs of groups
results = {}
for i in range(len(groups)):
    for j in range(i + 1, len(groups)):
        group_1 = group_data[groups[i]]
        print(groups[i]) 
        print(group_1)
        group_2 = group_data[groups[j]]
        print(groups[i]) 
        print(group_2)
        stat, p_value = mannwhitneyu(group_1, group_2, alternative='two-sided')
        results[f"{groups[i]} vs {groups[j]}"] = (stat, p_value)

results

Walking cane_rank
[2, 3, 4, 7, 6, 5, 1, 0, 8]
Walking cane_rank
[2, 4, 3, 8, 5, 6, 0, 1, 7]
Walking cane_rank
[2, 3, 4, 7, 6, 5, 1, 0, 8]
Walking cane_rank
[3, 2, 4, 5, 8, 6, 1, 0, 7]
Walking cane_rank
[2, 3, 4, 7, 6, 5, 1, 0, 8]
Walking cane_rank
[2, 3, 4, 1, 0, 6, 5, 8, 7]
Walking cane_rank
[2, 3, 4, 7, 6, 5, 1, 0, 8]
Walking cane_rank
[4, 3, 2, 5, 1, 8, 6, 0, 7]
Walker_rank
[2, 4, 3, 8, 5, 6, 0, 1, 7]
Walker_rank
[3, 2, 4, 5, 8, 6, 1, 0, 7]
Walker_rank
[2, 4, 3, 8, 5, 6, 0, 1, 7]
Walker_rank
[2, 3, 4, 1, 0, 6, 5, 8, 7]
Walker_rank
[2, 4, 3, 8, 5, 6, 0, 1, 7]
Walker_rank
[4, 3, 2, 5, 1, 8, 6, 0, 7]
Mobility scooter_rank
[3, 2, 4, 5, 8, 6, 1, 0, 7]
Mobility scooter_rank
[2, 3, 4, 1, 0, 6, 5, 8, 7]
Mobility scooter_rank
[3, 2, 4, 5, 8, 6, 1, 0, 7]
Mobility scooter_rank
[4, 3, 2, 5, 1, 8, 6, 0, 7]
Manual wheelchair_rank
[2, 3, 4, 1, 0, 6, 5, 8, 7]
Manual wheelchair_rank
[4, 3, 2, 5, 1, 8, 6, 0, 7]


{'Walking cane_rank vs Walker_rank': (40.5, 1.0),
 'Walking cane_rank vs Mobility scooter_rank': (40.5, 1.0),
 'Walking cane_rank vs Manual wheelchair_rank': (40.5, 1.0),
 'Walking cane_rank vs Motorized wheelchair_rank': (40.5, 1.0),
 'Walker_rank vs Mobility scooter_rank': (40.5, 1.0),
 'Walker_rank vs Manual wheelchair_rank': (40.5, 1.0),
 'Walker_rank vs Motorized wheelchair_rank': (40.5, 1.0),
 'Mobility scooter_rank vs Manual wheelchair_rank': (40.5, 1.0),
 'Mobility scooter_rank vs Motorized wheelchair_rank': (40.5, 1.0),
 'Manual wheelchair_rank vs Motorized wheelchair_rank': (40.5, 1.0)}